In [2]:
# Predicts diseases based on the symptoms entered and selected by the user.
# importing all necessary libraries
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, cross_val_score
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from itertools import combinations
from time import time
from collections import Counter
import operator
import math
from treatment import diseaseDetail
from sklearn.linear_model import LogisticRegression

warnings.simplefilter("ignore")

# New Section

In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/Manasa/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/Manasa/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/Manasa/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/Manasa/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/Manasa/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/Mana

True

In [4]:
# returns the list of synonyms of the input word from thesaurus.com (https://www.thesaurus.com/) and wordnet (https://www.nltk.org/howto/wordnet.html)
def synonyms(term):
    synonyms = []
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        row=container.find('div',{'class':'css-191l5o0-ClassicContentCard'})
        row = row.find_all('li')
        for x in row:
            synonyms.append(x.get_text())
    except:
        None
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)


In [5]:
# utlities for pre-processing
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
splitter = RegexpTokenizer(r'\w+')

In [6]:
# Load Dataset scraped from NHP (https://www.nhp.gov.in/disease-a-z) & Wikipedia
# Scrapping and creation of dataset csv is done in a separate program
df_comb = pd.read_csv("./Disease_Symptom_Dataset_For_All_Symptom_Subsets.csv") # All symptoms
df_indi = pd.read_csv("./Disease_Symptom_Dataset_For_Respective_Symptoms.csv") # Individual symptoms

X = df_comb.iloc[:, 1:]
Y = df_comb.iloc[:, 0:1]

In [7]:
lr = LogisticRegression()
lr = lr.fit(X, Y)
scores = cross_val_score(lr, X, Y, cv=5)

In [8]:
X = df_indi.iloc[:, 1:]
Y = df_indi.iloc[:, 0:1]

In [9]:
# List of symptoms
symptoms_dataset = list(X.columns)

In [10]:
# Taking symptoms from user as input 
user_symptoms = str(input("Please enter symptoms separated by comma(,):\n")).lower().split(',')
# Preprocessing the input symptoms
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)

Please enter symptoms separated by comma(,):
 fever,headache,stomach pain


In [11]:
# Taking each user symptom and finding all its synonyms and appending it to the pre-processed symptom string
user_symptoms = []
for user_sym in processed_user_symptoms:
    user_sym = user_sym.split()
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
# query expansion performed by joining synonyms found for each symptoms initially entered
print("After query expansion done by using the symptoms entered")
print(user_symptoms)

After query expansion done by using the symptoms entered
['fever feverishness febrility febricity pyrexia', 'vexation head ache cephalalgia headache worry concern', 'pain pain in the ass tummy painfulness stomach venter trouble stomach pain brook bother hurt annoyance tolerate stick out tum support put up pain sensation breadbasket endure painful sensation hurting pain in the neck belly anguish nuisance infliction suffer botheration stand ail abdomen abide digest bear']


In [12]:
# Loop over all the symptoms in dataset and check its similarity score to the synonym string of the user-input 
# symptoms. If similarity>0.5, add the symptom to the final list
found_symptoms = set()
for idx, data_sym in enumerate(symptoms_dataset):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5:
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [18]:
print(found_symptoms)

[]


In [15]:
# Print all found symptoms
print("Top matching symptoms from your search!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)
    
# Show the related symptoms found in the dataset and ask user to select among them
select_list = input("\nPlease select the relevant symptoms. Enter indices (separated-space):\n").split()

# Find other relevant symptoms from the dataset based on user symptoms based on the highest co-occurance with the
# ones that is input by the user
dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_indi[df_indi[symp]==1]['Disease_Name']))
   
for dis in dis_list:
    row = df_indi.loc[df_indi['Disease_Name'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and symptoms_dataset[idx] not in final_symp:
            counter_list.append(symptoms_dataset[idx])

Top matching symptoms from your search!
0 : fever
1 : stomach pain
2 : neck
3 : headache
4 : painful



Please select the relevant symptoms. Enter indices (separated-space):
 2


In [16]:
# Symptoms that co-occur with the ones selected by user              
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   
print(dict_symp_tup) 

[('arm', 1), ('back', 1), ('chest pain', 1), ('cold sweat', 1), ('cramped jaw', 1), ('feeling faint', 1), ('feeling tired', 1), ('nausea', 1), ('shortness breath', 1), ('stomach pain', 1)]


In [17]:
# Iteratively, suggest top co-occuring symptoms to the user and ask to select the ones applicable 
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCommon co-occuring symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = [] 


Common co-occuring symptoms:
0 : arm
1 : back
2 : chest pain
3 : cold sweat
4 : cramped jaw


Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
 1



Common co-occuring symptoms:
0 : feeling faint
1 : feeling tired
2 : nausea
3 : shortness breath
4 : stomach pain


Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
 3


In [16]:
# Create query vector based on symptoms selected by the user
print("\nFinal list of Symptoms that will be used for prediction:")
sample_x = [0 for x in range(0,len(symptoms_dataset))]
for val in final_symp:
    print(val)
    sample_x[symptoms_dataset.index(val)]=1


Final list of Symptoms that will be used for prediction:
fever
loss smell
coughing
sore throat


In [17]:
# Predict disease
lr = LogisticRegression()
lr = lr.fit(X, Y)
prediction = lr.predict_proba([sample_x])

In [18]:
k = 10
diseases = list(set(Y['Disease_Name']))
diseases.sort()
topk = prediction[0].argsort()[-k:][::-1]

In [19]:
print(f"\nTop {k} diseases predicted based on symptoms")
topk_dict = {}
# Show top 10 highly probable disease to the user.
for idx,t in  enumerate(topk):
    match_sym=set()
    row = df_indi.loc[df_indi['Disease_Name'] == diseases[t]].values.tolist()
    row[0].pop(0)

    for idx,val in enumerate(row[0]):
        if val!=0:
            match_sym.add(symptoms_dataset[idx])
    prob = (len(match_sym.intersection(set(final_symp)))+1)/(len(set(final_symp))+1)
    prob *= mean(scores)
    topk_dict[t] = prob
j = 0
topk_index_mapping = {}
topk_sorted = dict(sorted(topk_dict.items(), key=lambda kv: kv[1], reverse=True))
for key in topk_sorted:
  prob = topk_sorted[key]*100
  print(str(j) + " Disease name:",diseases[key], "\tProbability:",str(round(prob, 2))+"%")
  topk_index_mapping[j] = key
  j += 1

select = input("\nMore details about the disease? Enter index of disease or '-1' to discontinue and close the system:\n")
if select!='-1':
    dis=diseases[topk_index_mapping[int(select)]]
    print()
    print(diseaseDetail(dis))


Top 10 diseases predicted based on symptoms
0 Disease name: Common cold 	Probability: 75.04%
1 Disease name: Flu 	Probability: 75.04%
2 Disease name: Diphtheria 	Probability: 56.28%
3 Disease name: Strep throat 	Probability: 56.28%
4 Disease name: Pneumonia 	Probability: 56.28%
5 Disease name: Mononucleosis 	Probability: 56.28%
6 Disease name: Middle east respiratory syndrome coronavirus (mers‐cov) 	Probability: 56.28%
7 Disease name: Cough 	Probability: 56.28%
8 Disease name: Tonsillitis 	Probability: 56.28%
9 Disease name: Rubella 	Probability: 56.28%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
4

Pneumonia
Other names -  Pneumonitis 
Pronunciation -       /  nj  uː  ˈ  m  oʊ  n  i  ə  /    new- MOHN -ee-ə      
Specialty -  Pulmonology, Infectious disease 
Symptoms -  Cough, difficulty breathing, rapid breathing, fever   
Duration -  Few weeks   
Causes -  Bacteria, virus, aspiration   
Risk factors -  Cystic fibrosis, COPD, 